# Phase 2

In [2]:
pip uninstall -y numpy


Found existing installation: numpy 2.3.4
Uninstalling numpy-2.3.4:
  Successfully uninstalled numpy-2.3.4
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


In [1]:
pip install numpy==1.26.4 --force-reinstall --no-cache-dir


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/15.5 MB 3.7 MB/s eta 0:00:04
   ------ --------------------------------- 2.4/15.5 MB 5.4 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/15.5 MB 4.7 MB/s eta 0:00:03
   --------- ------------------------------ 3.7/15.5 MB 5.0 MB/s eta 0:00:03
   ------------ --------------------------- 5.0/15.5 MB 4.6 MB/s eta 0:00:03
   -------------- ------------------------- 5.5/15.5 MB 4.7 MB/s eta 0:00:03
   ----------------- ---------------------- 6.8/15.5 MB 4.4 MB/s eta 0:00:02
   ------------------ --------------------- 7.3/15.5 MB 4.2 MB/s eta 0:00:02
   -------------------- ------------------- 7.9/15.5 MB 4.2 MB/s eta 0:00:02
   --------------------- ------------------ 8.4/15.5 MB 4.0 MB/s eta 0:00:02
   ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.45.0 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [2]:
import numpy
import torch
print("NumPy version:", numpy.__version__)
print("PyTorch version:", torch.__version__)


NumPy version: 1.26.4
PyTorch version: 2.2.2+cpu


In [1]:
# Stage 2: Model Development - Text Generation using LSTM
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ma007\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\ma007\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\traitlets\config\application.py", line 1075, in launch

In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

import os
BASE_DIR = os.path.dirname(__file__)
TEXT_FILE = os.path.join(BASE_DIR, "pile_uncopyrighted_50MB.txt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


# Data Loading and Pre-processing

In [ ]:
#  LOADING DATA 
with open(TEXT_FILE, "r", encoding="utf-8") as f:
    text = f.read()


text = text[:500000]
print(f"Loaded text length: {len(text):,} characters")

#  CREATING CHAR MAPPING 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Unique characters:", vocab_size)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s): 
    return [stoi[c] for c in s]

def decode(l): 
    return ''.join([itos[i] for i in l])


Loaded text length: 500,000 characters
Unique characters: 91


# Dataset Class

In [ ]:
#  DATASET 
class TextDataset(Dataset):
    def __init__(self, data, seq_len=100):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        chunk = self.data[idx:idx+self.seq_len+1]
        input_seq = torch.tensor(chunk[:-1], dtype=torch.long)
        target_seq = torch.tensor(chunk[1:], dtype=torch.long)
        return input_seq, target_seq

encoded = encode(text)
dataset = TextDataset(encoded, seq_len=100)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


# LSTM Model

In [ ]:
#  MODEL 
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(CharLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

model = CharLSTM(vocab_size=vocab_size, embed_size=128, hidden_size=256, num_layers=2).to(device)


# Training Setup

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

EPOCHS = 5  

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        output, _ = model(inputs)
        loss = criterion(output.transpose(1, 2), targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f}")


Epoch [1/5] - Loss: 0.9473
Epoch [2/5] - Loss: 0.5545
Epoch [3/5] - Loss: 0.4718
Epoch [4/5] - Loss: 0.4387
Epoch [5/5] - Loss: 0.4194


# Text Generalization Function

In [ ]:
def generate_text(model, start_text="hello", length=300):
    model.eval()
    start_text = start_text.lower()  #  conversion to lowercase
    input_seq = torch.tensor(encode(start_text), dtype=torch.long).unsqueeze(0).to(device)
    hidden = None
    generated = list(start_text)

    for _ in range(length):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            last_logits = output[0, -1]
            probs = torch.nn.functional.softmax(last_logits, dim=0)
            idx = torch.multinomial(probs, num_samples=1).item()
            generated.append(itos[idx])
            input_seq = torch.tensor([[idx]], dtype=torch.long).to(device)

    return ''.join(generated)
print(generate_text(model, start_text="Once upon a time", length=500))


once upon a time Ġh u , Ġpod isag Ġgave , Ġbusiness Ġwere Ġfailured Ġformat . Ġyou Ġcan Ġbut Ġby Ġmap . Ġthese Ġmanner Ġwhile Ġdis ac char ides Ġand Ġhis Ġcoached Ġwith Ġa Ġtest Ġcareful Ġby Ġusing Ġmicro soft Ġtest Ġmanager Ġcreates Ġand Ġman gard . Ġwe Ġhave Ġof Ġmeasures Ġwere Ġrevealed . Ġsuccess Ġand Ġmarkup Ġmolder Ġfrom Ġhome Ġin Ġthe Ġfr c Ġnoted Ġthat Ġ50 Ġof Ġk pm gs Ġft se Ġ350 Ġaudits Ġfailed Ġto Ġremove Ġbe ijing Ġphbillar Ġprosect Ġno . Ġthe Ġ12 pot ty es Ġ Ġt Ġt Ġt j . Ġget Ġtot al ies Ġmarker Ġs


In [ ]:
#  GENERATE TEXT 
def generate_text(model, start_text="hello", length=300):
    model.eval()
    start_text = start_text.lower()   # 🔥 convert to lowercase to match vocab
    input_seq = torch.tensor(encode(start_text), dtype=torch.long).unsqueeze(0).to(device)
    hidden = None
    generated = list(start_text)

    for _ in range(length):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            probs = torch.softmax(output[:, -1, :], dim=-1).detach().cpu().numpy().ravel()
            next_idx = np.random.choice(len(probs), p=probs)
            next_char = itos[next_idx]
            generated.append(next_char)
            input_seq = torch.tensor([[next_idx]], dtype=torch.long).to(device)

    return ''.join(generated)

# Test after training
print(generate_text(model, start_text="Once upon a time", length=500))


once upon a time Ġhyp ox amenied Ġof Ġhtml wont Ġdefenced Ġenvelope Ġand Ġgravity Ġwaves Ġin Ġthe Ġsun Ġmajor Ġon Ġour Ġpetition Ġfor Ġdisturbances Ġsuch Ġas Ġthe Ġan is otide Ġfunction . Ġwhat Ġdo Ġyou Ġthink Ġis Ġalso Ġoften Ġused Ġto Ġincrease Ġherogid Ġtype Ġwith Ġservers , Ġincluding Ġwas Ġbecause Ġit Ġnotes Ġor Ġoil Ġhy row Ġfor Ġintervening Ġdenical Ġhardly , Ġthe Ġoutput Ġfile , Ġwashing ton , Ġd 0 d 15 a Ġ Ġinc ., Ġ51 Ġfr and i od
party Ġthe Ġisolated Ġnuscle Ġcenter Ġin Ġrecent Ġhydrogen Ġbygara Ġto Ġ


# Saving Model

In [11]:
MODEL_PATH = os.path.join(BASE_DIR, "char_lstm_model.pth")
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved at {MODEL_PATH}")


Model saved at C:\Users\ma007\OneDrive\Desktop\Buildables_Project3\char_lstm_model.pth
